GuideLine：
1. 数据增强收益更大
    - 半监督：将预测概率很大的标记，作为训练数据再加入
    - 第三方成熟的nlp包的识别结果，作为特征加入

2. 可以使用规则的标签，可以直接使用规则方式

3. 不同标记体系可以尝试

# 解压数据

In [1]:
!ls ../data/

Category_info.csv
test_data
train.txt
train_data
train_label
train_new
val.txt
val_new


In [2]:
# !ls ../data/train_data

In [3]:
!head ../data/train_data/0.txt

鑹烘湳鏄�鐩稿悓鐨勶紝闊充箰缇庢湳浣撹偛涓夋牱閮芥槸鑹烘湳銆傦紝涓夋牱鑹烘湳閮芥槸闈犳劅瑙夌殑銆傛劅瑙夊ソ鐜╄捣鏉ュ氨寰堣交鏉撅紝鎵�浠ュ彨鍋氱帺鑹烘湳銆傛病鎰熻�夋壘涓嶅埌鍖楃殑骞茶剢鍒�鐜╀簡锛侊紝棣欐腐鐢靛奖鍥借��閰嶉煶鍚嶅�跺懆鎬濆钩锛屼唬琛ㄤ綔鏈塗VB銆婁笂娴锋哗銆嬪懆娑﹀彂绛�


In [4]:
!head ../data/train_label/0.csv

锘縄D,Category,Pos_b,Pos_e,Privacy
0,position,66,75,棣欐腐鐢靛奖鍥借��閰嶉煶鍚嶅��
0,name,76,78,鍛ㄦ�濆钩
0,movie,87,91,銆婁笂娴锋哗銆�
0,name,92,94,鍛ㄦ鼎鍙�
0,organization,84,86,TVB


In [5]:
# !unzip ../data/train.zip -d ../data/train
# !unzip ../data/test.zip -d ../data/test

In [6]:
# !ls ../data/train_data/

In [7]:
# !ls ../data/test_data/

In [8]:
# !pip install tqdm

# 读取数据

## 训练数据标记

In [1]:
import os
import pandas as pd
from tqdm import tqdm

In [2]:
def get_df_label(label_path):
    "单个标签文件读取"
    return pd.read_csv(label_path)

In [3]:
def get_list_data(data_path):
    "文本文件转为 单个字组成的list"
    with open(data_path, 'r', encoding='utf-8') as f:
        line = ''.join(f.readlines())
        list_data = list(line)
    return list_data

### IOBES

In [4]:
def get_BIOES(list_data, df_label):
    "将单字和对应ner标记，一一对应，标记体系为BIOES"
    list_label = ['O'] * len(list_data)
    for index, d in df_label.iterrows():
        ID = d['ID']
        Category = d['Category']
        Pos_b = d['Pos_b']
        Pos_e = d['Pos_e']

        if Pos_b == Pos_e:  # 单字
            list_label[Pos_b] = 'S_' + Category
        elif Pos_e - Pos_b == 1:  # 双字
            list_label[Pos_b] = 'B_' + Category
            list_label[Pos_e] = 'E_' + Category
        else: # 多字
            try:
                list_label[Pos_b] = 'B_' + Category
                list_label[Pos_e] = 'E_' + Category
                for pos_i in range(Pos_b + 1, Pos_e):
                    list_label[pos_i] = 'I_' + Category
            except:
                print(ID)
    return [(ID, data, label) for data, label in zip(list_data, list_label)]

### IOB

In [5]:
# TODO
def get_IOB(list_data, df_label):
    "将单字和对应ner标记，一一对应，标记体系为BIOES"
    list_label = ['O'] * len(list_data)
    for index, d in df_label.iterrows():
        ID = d['ID']
        Category = d['Category']
        Pos_b = d['Pos_b']
        Pos_e = d['Pos_e']

        if Pos_b == Pos_e:  # 单字
            list_label[Pos_b] = 'B_' + Category
        else: # 多字
            try:
                list_label[Pos_b] = 'B_' + Category
                for pos_i in range(Pos_b + 1, Pos_e + 1):
                    list_label[pos_i] = 'I_' + Category
            except:
                print(ID)
    return [(ID, data, label) for data, label in zip(list_data, list_label)]

In [6]:
def get_data_label(path_dir_data, path_dir_label):
    "标记并输出文件"
    # 训练数据集
    list_IOB = []
    # 获取训练集 ID
    list_data_ID = [
        file_name.split('.')[0] for file_name in os.listdir(path_dir_data)
        if '.txt' in file_name
    ]
    list_label_ID = [
        file_name.split('.')[0] for file_name in os.listdir(path_dir_label)
        if '.csv' in file_name
    ]

    # 循环训练集 ID
    for ID in tqdm(set(list_data_ID) & set(list_label_ID)):
        # 获取词和标签地址
        data_path = os.path.join(path_dir_data, ID + '.txt')
        label_path = os.path.join(path_dir_label, ID + '.csv')

        # 获取词和标签数据
        list_data = get_list_data(data_path)
        df_label = get_df_label(label_path)

        # IOB 进行标注
        IOB = get_IOB(list_data, df_label)

        # 加入训练数据集
        list_IOB += [IOB]

    return list_IOB

In [7]:
path_dir_data = '../data/train_data/'
path_dir_label = '../data/train_label/'

list_IOB_train = get_data_label(path_dir_data,path_dir_label)

100%|████████████████████████████████████████████████████████████| 2515/2515 [00:31<00:00, 79.06it/s]


In [8]:
list_IOB_train[0]

[(1561, '5', 'O'),
 (1561, '轮', 'O'),
 (1561, '不', 'O'),
 (1561, '胜', 'O'),
 (1561, '的', 'O'),
 (1561, '亚', 'B_organization'),
 (1561, '特', 'I_organization'),
 (1561, '兰', 'I_organization'),
 (1561, '大', 'I_organization'),
 (1561, '本', 'O'),
 (1561, '场', 'O'),
 (1561, '迎', 'O'),
 (1561, '来', 'O'),
 (1561, '多', 'B_name'),
 (1561, '尼', 'I_name'),
 (1561, '和', 'O'),
 (1561, '主', 'O'),
 (1561, '力', 'O'),
 (1561, '中', 'O'),
 (1561, '场', 'O'),
 (1561, '阿', 'B_name'),
 (1561, '斯', 'I_name'),
 (1561, '森', 'I_name'),
 (1561, '蒂', 'I_name'),
 (1561, '斯', 'I_name'),
 (1561, '的', 'O'),
 (1561, '复', 'O'),
 (1561, '出', 'O'),
 (1561, '，', 'O'),
 (1561, '与', 'O'),
 (1561, '上', 'O'),
 (1561, '轮', 'O'),
 (1561, '惜', 'O'),
 (1561, '败', 'O'),
 (1561, '给', 'O'),
 (1561, '佛', 'B_organization'),
 (1561, '罗', 'I_organization'),
 (1561, '伦', 'I_organization'),
 (1561, '萨', 'I_organization'),
 (1561, '相', 'O'),
 (1561, '比', 'O'),
 (1561, '阵', 'O'),
 (1561, '容', 'O'),
 (1561, '更', 'O'),
 (1561, '加', 'O'),
 (1561

NOTE：IOB标记的数据模型还未跑，IOBES已完成模型训练

### 输出标记文件

In [9]:
def get_data_label(path_dir_data, path_dir_label):
    "标记并输出文件"
    # 训练数据集
    list_BIOES = []
    # 获取训练集 ID
    list_data_ID = [
        file_name.split('.')[0] for file_name in os.listdir(path_dir_data)
        if '.txt' in file_name
    ]
    list_label_ID = [
        file_name.split('.')[0] for file_name in os.listdir(path_dir_label)
        if '.csv' in file_name
    ]

    # 循环训练集 ID
    for ID in tqdm(set(list_data_ID) & set(list_label_ID)):
        # 获取词和标签地址
        data_path = os.path.join(path_dir_data, ID + '.txt')
        label_path = os.path.join(path_dir_label, ID + '.csv')

        # 获取词和标签数据
        list_data = get_list_data(data_path)
        df_label = get_df_label(label_path)

        # BIOES 进行标注
        BIOES = get_BIOES(list_data, df_label)

        # 加入训练数据集
        list_BIOES += [BIOES]

    return list_BIOES

In [12]:
path_dir_data = '../data/train_data/'
path_dir_label = '../data/train_label/'

In [11]:
list_BIOES_train = get_data_label(path_dir_data,path_dir_label)

100%|███████████████████████████████████████████████████████████| 2515/2515 [00:10<00:00, 229.01it/s]


In [12]:
list_BIOES_train[0]

[(1561, '5', 'O'),
 (1561, '轮', 'O'),
 (1561, '不', 'O'),
 (1561, '胜', 'O'),
 (1561, '的', 'O'),
 (1561, '亚', 'B_organization'),
 (1561, '特', 'I_organization'),
 (1561, '兰', 'I_organization'),
 (1561, '大', 'E_organization'),
 (1561, '本', 'O'),
 (1561, '场', 'O'),
 (1561, '迎', 'O'),
 (1561, '来', 'O'),
 (1561, '多', 'B_name'),
 (1561, '尼', 'E_name'),
 (1561, '和', 'O'),
 (1561, '主', 'O'),
 (1561, '力', 'O'),
 (1561, '中', 'O'),
 (1561, '场', 'O'),
 (1561, '阿', 'B_name'),
 (1561, '斯', 'I_name'),
 (1561, '森', 'I_name'),
 (1561, '蒂', 'I_name'),
 (1561, '斯', 'E_name'),
 (1561, '的', 'O'),
 (1561, '复', 'O'),
 (1561, '出', 'O'),
 (1561, '，', 'O'),
 (1561, '与', 'O'),
 (1561, '上', 'O'),
 (1561, '轮', 'O'),
 (1561, '惜', 'O'),
 (1561, '败', 'O'),
 (1561, '给', 'O'),
 (1561, '佛', 'B_organization'),
 (1561, '罗', 'I_organization'),
 (1561, '伦', 'I_organization'),
 (1561, '萨', 'E_organization'),
 (1561, '相', 'O'),
 (1561, '比', 'O'),
 (1561, '阵', 'O'),
 (1561, '容', 'O'),
 (1561, '更', 'O'),
 (1561, '加', 'O'),
 (1561

In [39]:
# for item in list_BIOES_train:
#     if item[0][0]==2162:
#         print(item)

## 测试数据标记

In [13]:
def get_data(path_dir_data):

    # 测试数据集
    list_toPredict = []
    # 获取测试集 ID
    list_data_ID = [
        file_name.split('.')[0] for file_name in os.listdir(path_dir_data)
        if '.txt' in file_name
    ]

    # 循环测试集 ID
    for ID in tqdm(set(list_data_ID)):
        # 获取词
        data_path = os.path.join(path_dir_data, ID + '.txt')

        # 获取词
        list_data = get_list_data(data_path)

        # 无标注
        toPredict = [(ID, data, 'N') for data in list_data]

        # 加入训练数据集
        list_toPredict += [toPredict]

    return list_toPredict

In [11]:
path_dir_data = '../data/test_data'

In [15]:
list_toPredict_test = get_data(path_dir_data)

100%|███████████████████████████████████████████████████████████| 3956/3956 [00:13<00:00, 285.22it/s]


In [16]:
list_toPredict_test[0]

[('1561', '宁', 'N'),
 ('1561', '可', 'N'),
 ('1561', '认', 'N'),
 ('1561', '错', 'N'),
 ('1561', '也', 'N'),
 ('1561', '不', 'N'),
 ('1561', '要', 'N'),
 ('1561', '说', 'N'),
 ('1561', '谎', 'N'),
 ('1561', '，', 'N'),
 ('1561', '因', 'N'),
 ('1561', '为', 'N'),
 ('1561', '认', 'N'),
 ('1561', '错', 'N'),
 ('1561', '只', 'N'),
 ('1561', '是', 'N'),
 ('1561', '面', 'N'),
 ('1561', '子', 'N'),
 ('1561', '问', 'N'),
 ('1561', '题', 'N'),
 ('1561', '，', 'N'),
 ('1561', '而', 'N'),
 ('1561', '说', 'N'),
 ('1561', '谎', 'N'),
 ('1561', '却', 'N'),
 ('1561', '要', 'N'),
 ('1561', '付', 'N'),
 ('1561', '出', 'N'),
 ('1561', '长', 'N'),
 ('1561', '期', 'N'),
 ('1561', '包', 'N'),
 ('1561', '装', 'N'),
 ('1561', '成', 'N'),
 ('1561', '本', 'N'),
 ('1561', ';', 'N'),
 ('1561', '宁', 'N'),
 ('1561', '可', 'N'),
 ('1561', '挨', 'N'),
 ('1561', '批', 'N'),
 ('1561', '也', 'N'),
 ('1561', '不', 'N'),
 ('1561', '要', 'N'),
 ('1561', '内', 'N'),
 ('1561', '疚', 'N'),
 ('1561', '，', 'N'),
 ('1561', '因', 'N'),
 ('1561', '为', 'N'),
 ('1561', '挨'

# 查看数据

## 获取数据

In [4]:
def get_dataframe(path_dir_data):
    # 获取的数据
    list_ID = []
    list_data = []

    # 获取ID
    list_data_ID = [
        file_name.split('.')[0] for file_name in os.listdir(path_dir_data)
        if '.txt' in file_name
    ]
    # 循环ID
    for ID in tqdm(set(list_data_ID)):
        # 获取词
        data_path = os.path.join(path_dir_data, ID + '.txt')
        with open(data_path, 'r', encoding='utf-8') as f:
            line = ''.join(f.readlines())
        # 获取词
        list_ID += [ID]
        list_data += [line]
    # dataframe
    df_data = pd.DataFrame()
    df_data['ID'] = list_ID
    df_data['data'] = list_data
    return df_data

In [5]:
path_dir_data_train = '../data/train_data/'
path_dir_data_test = '../data/test_data/'

In [6]:
df_train = get_dataframe(path_dir_data_train)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2515/2515 [00:10<00:00, 244.58it/s]


In [7]:
df_test = get_dataframe(path_dir_data_test)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3956/3956 [00:14<00:00, 280.29it/s]


In [28]:
df_train

,ID,data
0,2345,综合6项指标，最后评出工行综合服务最好，北京银行最差。北京银行之所以得分最低，主因是还处于起...
1,84,实录：丽泽金融商务区发展高峰论坛（组图）也不曾看到这两家网站提起过，因此我们可以断定这个倒计...
2,1482,此外，就是否将美军普天间机场的原定搬迁目的地名护市的设施建设费等288亿日元（约合21.Oa...
3,2092,代言和授权请联系fanstang@fanstang.com;AdamLevine，Maroo...
4,1054,德军的科学家们正在进行秘密实验，一个比唤醒僵尸大军还要恐怖的实验计划。
...,...,...
2510,2143,神愛世人！我愛你！网站：ericsuen.com或er16.com或lo-suen.hk工作...
2511,1915,最好的例子就是星际国家冠军林耀焕（Slayers_BoxeR）。“他真的非常出名。
2512,545,大家好！我叫DomePakornlam。你們可以在IG:domepakornlam，icon...
2513,1385,入圍第十八屆金曲獎最佳客語演唱獎、第四屆母語原創大賽最佳表演獎。相關洽詢：+88673802...


In [29]:
df_test

,ID,data
0,2345,关于刀锋女皇萨拉·凯瑞甘故事，你最好的了解方式就是阅读《利伯蒂的远征》和《刀锋女皇》一位不愿...
1,84,我是柯英豪，来自美国南部的德克萨斯州。现任美国驻华大使馆新闻发言人一职。爱好包括旅游，美食和...
2,2752,简单而安全的滑索让你全身心投入到高空中，拿出勇气，展开手臂，用超人的姿态来征服此刻的卢霍山。
3,1482,网易在内部邮件中谈及此前魔兽开服所遇到的困难时这样表示。
4,2092,郑大北门西20米二楼）的七彩商都网吧。有兴趣与高手一决胜负的电竞爱好者们欢迎踊跃报名。
...,...,...
3951,3830,【钢丝善行团管理委员会委员】政务名博、新浪微公益爱心团成员、新浪微博社区委员会专家成员。ht...
3952,3212,80后作家，出版《玄机》《战狼》《墓闻录》《天佛之眼》《搜异笔记》《死神十字》《神秘事件调查...
3953,545,中国摄影家协会会员、国际摄影协会会员、香港国际摄影杂志社记者、新华社签约摄影师。博客：htt...
3954,1385,傲娇的伪萌者，爱魔力，爱西索，爱东方，COS玩的是开心~我不是人民币不求人见人爱~，CCAS...


## 获取标签

In [8]:
def get_label(path_dir_data):
    # 获取的数据
    list_df_label = []
    
    # 获取ID
    list_label_ID = [file_name.split('.')[0] for file_name in os.listdir(path_dir_label) if '.csv' in file_name]
    # 循环ID
    for ID in tqdm(set(list_label_ID)):
        # 获取标签
        label_path = os.path.join(path_dir_label,ID+'.csv')
        df_label = get_df_label(label_path)
        list_df_label += [df_label]
        
    # dataframe
    df_label = pd.concat(list_df_label)
    return df_label

In [13]:
df_label_train = get_label('../data/train_label/')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2515/2515 [00:11<00:00, 211.09it/s]


In [33]:
df_label_train.head()

,ID,Category,Pos_b,Pos_e,Privacy
0,2345,company,11,12,工行
1,2345,company,20,23,北京银行
2,2345,company,27,30,北京银行
3,2345,address,52,61,三街玉苍路路口红绿灯
4,2345,book,132,135,草间弥生


In [34]:
set(df_label_train.Category)

{'QQ',
 'address',
 'book',
 'company',
 'email',
 'game',
 'government',
 'mobile',
 'movie',
 'name',
 'organization',
 'position',
 'scene',
 'vx'}

## 聚合不同类型标签查看

In [17]:
# 分别聚合所有标签包含的实体
df_label_train_Category = df_label_train.groupby(['Category'])['Privacy'].agg(lambda x: ' | '.join([str(i) for i in list(x)])).reset_index()

In [18]:
df_label_train_Category

,Category,Privacy
0,QQ,1143175993 | 2308713823 | 783594553 | 14705257...
1,address,库兰达 | 香格里拉县城 | 香港 | 香港 | 台湾 | 沃尔库塔劳工集中营 | 恒晟超市...
2,book,《了不起的盖茨比》 | 《经济参考报》 | 《电脑游戏攻略》 | 《北京市经济适用住房销售许...
3,company,招行 | 北京骑士美苑影视文化传媒有限公司 | 中行 | 宝钢 | 新浪乐居 | 新浪网联 ...
4,email,1213785766@qq.com | tvbbecky@gmail.com | young...
5,game,魔兽争霸 | war3 | war3 | dota1 | 战争世界》 | 《终结者：救世主》...
6,government,美军 | 美国陆军 | 北京市教工委 | 市教委 | 俄海军 | 陕西省安康市旬阳县人民法院...
7,mobile,18601219959 | 13501058727 | 852-23352105 | +86...
8,movie,《你是我的命运》 | 《黄金新娘》 | 《无极》 | 《夜宴》 | 《赵氏孤儿》 | 《建国...
9,name,孔贤珠 | 孔贤珠 | 王莹小姐 | 大鹏， | 程寅 | tvbbecky | 乐瞳 | ...


In [19]:
df_label_train_Category.Privacy[8]

'《你是我的命运》 | 《黄金新娘》 | 《无极》 | 《夜宴》 | 《赵氏孤儿》 | 《建国大业》 | 《建党伟业》 | 《和平年代》 | 《新四军》 | 《国家干部》 | 《高地》 | 《高粱红了》 | 《战争目光》 | 《粟裕大将》 | 《攻心》 | 《追爱》 | 《女汤》 | 《袋鼠男人》 | 《追爱》 | 《女汤》 | 《推手》 | （Wheelman） | 《汽车驾驶员》 | 天龙八部 | 华丽上班族 | 说好的幸福 | 《辐射》 | 《怒潮》 | 《半个纳尔逊》 | 《波斯王子：时之沙》 | 《越狱》 | 《男人帮》 | 《北京青年》 | 《梁祝》 | 《隐形人》 | 追梦者》 | 《五号战星》 | 《圣女魔咒》 | 《赎罪》（atonement） | 《唐山大地震》 | 《决胜21点》 | 《爱在瘟疫蔓延时》 | loveinthetimeofcholera | 《完美假日》 | 《美容大王》 | 《步步惊心》 | 《新水浒传》 | 《姨妈的后现代生活》 | 《涂鸦的婚事》 | 【搜神记 | 【赤焰战场 | 《长江七号》 | 《长江七号》 | 《西施》 | 《这种爱》 | 红楼梦 | 《白鹿原》 | 《绝代商骄》 | 《幕后大老爷》 | 《非常岳母》 | 《革命到底》 | 变形金刚》 | 《婚礼之后》 | 《回归》 | 《爱情大师》 | 《书剑》 | 《小菊的春天》 | ≪书剑≫ | ≪小菊的春天秋天≫ | ≪千山暮雪≫ | 《丝丝心动》 | 《小菊的秋天》 | 《千山暮雪》 | 《京華煙雲》 | 《在水一方》 | 《又見阿郎》 | 《选举夜》 | 但丁的地狱》 | 《死亡空间》 | 醒世媳婦 | 台灣水滸傳 | 乞丐郎君千金女 | 世間路 | 真情滿天下 | 天下父母心 | 刁蠻新娘 | 桃花劫牽手 | 我家有喜 | 《黑蛋 | 《黄小鸿之狮舞骄阳》 | 《都市有情天》 | 《天衣行动》 | 《英雄无敌》 | 《纳尼亚传奇》 | 《初恋50次》 | 《辣妈正传》 | 《一仆二主》 | 《媳妇的美好时代》 | 《幸福已逝》 | 《超人》 | 《绝代佳人》 | 《兰花花》 | 《我的青春谁做主》 | 《疯狂德州》 | 《疯狂德州》 | 《永不回头》 | 《永不回头》 | 《泰坦尼克号》 | 《葵花子》 | 《天脊》 | 随缘行恶 | RandomAct

In [44]:
df_label_train_Category.Privacy[0]

'33370722 496049279 2450006972 1827967769 776979804 89359735 1787817738 1941600020 527622886 18618193311 415392980 826929452 924969718 371451138 160308912 14705257 326597719 1143175993 783594553 229215757 114281231 449801320 574785760 490401933 130238288 2308713823 53344829 232297193'

In [48]:
df_label_train_Category.Privacy[4]

"fanstang@fanstang.com angelinazhu@163.com abbyeung@gmail.com fudoushinn@hotmail.com hejingxinxiang@163.com jimmylin@jimmylin.a haowenjing718@126.com wuyuemail@126.com GaryCheung@ gcheung1973@yahoo.com wujing@missworld.cn fanstang@fanstang.com houxian_work@thmusic.com.cn baotianmao@hotmail.com jjrjiangyue@qq.com Ld6919@126.com rickychiu2004@yahoo.com.hk daijun_studio@126.com pancy8888@yahoo.com.hk he15012501083@gmail.com montager@foxmail.com 'pamela@dreamstardom.com zxl129@vip.sina.com 1213785766@qq.com info@nova-ent.com jjay999@163.com pccwman@hotmail.com saent@netvigator.com bianjuyuxiaoqian@sina.cn bianjuyuxiaoqian@vip.126.com ycwwhy@sina.com primetime@ptmedia.asia rain_ho2656@yahoo.com.cn nancykou0522@yahoo.com.tw wangwuli@vip.126.com tt@skykee.com info@wowmusic.hk lingying@188.com manager@snh48.com eleelegroup@gmail.com artiste.harmonic@gmail.com jessicafplydia@yahoo.com.hk. cywh88888@vip.126.com miki1234567@tom.com anjon0731@sina.com viki077@hotmail.com chrissyliu@mac.com 9867288

In [49]:
df_label_train_Category.Privacy[7]

'13801318005 18600608880 +85223352109 +886963110290 13521243618 +886-936-092-241 +886-952-199-882 852﹣23352164 010-6362212113910492247 +86-18601200718 （852）66088899（86）13818199118 18601012468 zzz813222 18500057591 13501379817 13910735585 13818958565 13817519526 18621199838 010﹣85570230 13810105207 0085290544220 18601160682 86-150-1250-1083 18953120001 +60128816178 18601219959 18611545632 13691110092 15232185888 00886928592269 13439591797 13261538083 139-1102-7909 69010858 13910663803 13910239835 18613038899 347-738-8723 +886935516828 18651870177 13776659833 18628169167 15828284929 18611797612 +852-37411033 18601126676 15001267764 13520605827 13552380800 13910819844 (02)2781-3888#51 886921-379-135( +886-2-28928639 886-989-264-950 137-0107-2631 139-010-66161 158-1121-2183 +86137-5787-9227 886-2-2753-1981 886926929796 13910008890 18611811168 13810737919 886-2-2738-7077 1827967769 886-2369-9886#585 +886910741038 13611356939 85172333-606 13439992202 18977110565 +886-2-66365725 15013488725 1

In [51]:
df_label_train_Category.Privacy[13]

'zhangzhizheng-zzz rachel-213 montager mengmeng6570 wwjshow liushasha319439 1827967769 13426245800 hongshui001 hongshui007 zhuzishanmisstin sjyy98 zhuzhiyi91510 waveapp songxiaobo tianlangyinyue 57150788 57150788 lily潘小芬'

In [ ]:
df_label_train_Category.Privacy[0]

In [37]:
df_label_train_Category.to_csv('../data/Category_info.csv',
                               header=True,
                               index=False,
                               encoding="utf-8")

In [40]:
# !head ../data/Category_info.csv

# 正则表达式

In [52]:
import re

In [58]:
set(df_label_train_Category['Category'])

{'QQ',
 'address',
 'book',
 'company',
 'email',
 'game',
 'government',
 'mobile',
 'movie',
 'name',
 'organization',
 'position',
 'scene',
 'vx'}

### 电话匹配

In [95]:
# 匹配电话
phoneRegex = re.compile(r'''(
    (13[0-9]|14[5|7]|15[0-9]|18[0-9])(\d|-){8,10}|
    \+?\d{3}-\d{8}|\d{4}-\d{7,8}|
    \+\d{11,12}
)''', re.VERBOSE)           # 如上，传入参数 re.VERBOSE 可以给正则表达式添加注释，详见附录

In [59]:
df_label_train_Category[df_label_train_Category['Category']=='mobile']

,Category,Privacy
7,mobile,13801318005 18600608880 +85223352109 +88696311...


In [60]:
text_phone = df_label_train_Category[df_label_train_Category['Category']=='mobile']['Privacy'].values[0]

In [93]:
text_phone

'13801318005 18600608880 +85223352109 +886963110290 13521243618 +886-936-092-241 +886-952-199-882 852﹣23352164 010-6362212113910492247 +86-18601200718 （852）66088899（86）13818199118 18601012468 zzz813222 18500057591 13501379817 13910735585 13818958565 13817519526 18621199838 010﹣85570230 13810105207 0085290544220 18601160682 86-150-1250-1083 18953120001 +60128816178 18601219959 18611545632 13691110092 15232185888 00886928592269 13439591797 13261538083 139-1102-7909 69010858 13910663803 13910239835 18613038899 347-738-8723 +886935516828 18651870177 13776659833 18628169167 15828284929 18611797612 +852-37411033 18601126676 15001267764 13520605827 13552380800 13910819844 (02)2781-3888#51 886921-379-135( +886-2-28928639 886-989-264-950 137-0107-2631 139-010-66161 158-1121-2183 +86137-5787-9227 886-2-2753-1981 886926929796 13910008890 18611811168 13810737919 886-2-2738-7077 1827967769 886-2369-9886#585 +886910741038 13611356939 85172333-606 13439992202 18977110565 +886-2-66365725 15013488725 1

In [97]:
[whole for whole, g1, g2 in phoneRegex.findall(text_phone)]

['13801318005',
 '18600608880',
 '+85223352109',
 '+886963110290',
 '13521243618',
 '010-63622121',
 '13910492247',
 '18601200718',
 '13818199118',
 '18601012468',
 '18500057591',
 '13501379817',
 '13910735585',
 '13818958565',
 '13817519526',
 '18621199838',
 '13810105207',
 '18601160682',
 '150-1250-1083',
 '18953120001',
 '+60128816178',
 '18601219959',
 '18611545632',
 '13691110092',
 '15232185888',
 '13439591797',
 '13261538083',
 '139-1102-7909',
 '13910663803',
 '13910239835',
 '18613038899',
 '+886935516828',
 '18651870177',
 '13776659833',
 '18628169167',
 '15828284929',
 '18611797612',
 '+852-37411033',
 '18601126676',
 '15001267764',
 '13520605827',
 '13552380800',
 '13910819844',
 '137-0107-2631',
 '139-010-66161',
 '158-1121-2183',
 '137-5787-9227',
 '13910008890',
 '18611811168',
 '13810737919',
 '+886910741038',
 '13611356939',
 '13439992202',
 '18977110565',
 '15013488725',
 '15920766912',
 '18611801269',
 '18665092260',
 '+886287713553',
 '138-0121-2506',
 '18911175595

### QQ匹配

In [135]:
qqRegex = re.compile(r'''(
    \b[1-9][0-9]{4,8}\b
)''', re.VERBOSE) 

In [136]:
qq_text = df_label_train_Category[df_label_train_Category['Category']=='QQ'].Privacy.values[0]

In [137]:
qq_text

'33370722 496049279 2450006972 1827967769 776979804 89359735 1787817738 1941600020 527622886 18618193311 415392980 826929452 924969718 371451138 160308912 14705257 326597719 1143175993 783594553 229215757 114281231 449801320 574785760 490401933 130238288 2308713823 53344829 232297193'

In [138]:
qqRegex.findall(qq_text)

['33370722',
 '496049279',
 '776979804',
 '89359735',
 '527622886',
 '415392980',
 '826929452',
 '924969718',
 '371451138',
 '160308912',
 '14705257',
 '326597719',
 '783594553',
 '229215757',
 '114281231',
 '449801320',
 '574785760',
 '490401933',
 '130238288',
 '53344829',
 '232297193']

NOTE: 先匹配电话，标记之后，再匹配qq，匹配时注意加上条件判断，已经被标记的电话，部分会符合qq的pattern

### 邮件地址

In [76]:
# 匹配邮件地址
emailRegex = re.compile(r'''(
    [a-zA-Z0-9._%+-]+           # email-username
    @
    [a-zA-Z0-9.-]+              # domain-name
    \.[a-zA-Z]{2,4}            # dot-something
)''', re.VERBOSE)

In [73]:
text = df_test[df_test['ID']=='2984'].data.values[0]

In [74]:
text

'工作邮箱联系：33119699@qq.com我的淘宝店:http://baby-amelie.taobao.com/，内地组合至上励合成员刘洲成'

In [77]:
emailRegex.findall(text)

['33119699@qq.com']

# 传统NER

## CRF

In [142]:
# !pip install sklearn_crfsuite

In [143]:
import sklearn
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

### 第三方工具特征处理

In [17]:
import os
import re

from pyltp import Segmentor
from pyltp import Postagger
from pyltp import Parser
from pyltp import NamedEntityRecognizer
from pyltp import SentenceSplitter

LTP_DATA_DIR = 'D:/ProgramData/nlp_package/ltp_v34'

class Ltp:
    "https://pyltp.readthedocs.io/zh_CN/latest/"
    def __init__(self, seg=True, pos=False, ner=False, parse=False,
                 seg_lexicon_path=None, pos_lexicon_path=None):
        cws_model_path = os.path.join(LTP_DATA_DIR, 'cws.model')  # 分词模型路径，模型名称为`cws.model`
        pos_model_path = os.path.join(LTP_DATA_DIR, 'pos.model')
        ner_model_path = os.path.join(LTP_DATA_DIR, 'ner.model')  # 命名实体识别模型路径
        par_model_path = os.path.join(LTP_DATA_DIR, 'parser.model')  # 依存句法分析模型

        if seg:
            self.segmentor = Segmentor()               #分词
            if seg_lexicon_path:
                self.segmentor.load_with_lexicon(cws_model_path,
                                                 seg_lexicon_path)
            else:
                self.segmentor.load(cws_model_path)

        if pos:
            # 输入分词结果
            self.postagger = Postagger()               #词性标注
            self.postagger.load(pos_model_path)
            if pos_lexicon_path:
                self.postagger.load_with_lexicon(pos_model_path,
                                                 pos_lexicon_path)
            else:
                self.postagger.load(pos_model_path)

        if ner:
            # 输入分词和标注结果
            self.ner = NamedEntityRecognizer()  #命名主体识别
            self.ner.load(ner_model_path)

        if parse:
            # 输入分词和标注结果
            self.parser = Parser()                     #依存分析
            self.parser.load(par_model_path)

    def release(self):
        try:
            self.segmentor.release()
            self.postagger.release()
            self.ner.release()
            self.parser.release()
        except AttributeError:
            pass

    def __del__(self):
        self.release()

#### 示例

In [18]:
ltp = Ltp(pos=True, ner=True, parse=True)

In [26]:
df_train.data[0]

'5轮不胜的亚特兰大本场迎来多尼和主力中场阿斯森蒂斯的复出，与上轮惜败给佛罗伦萨相比阵容更加完整。'

In [160]:
cut_words = ltp.segmentor.segment(df_train.data[0])
postags = ltp.postagger.postag(list(cut_words))
netags = ltp.ner.recognize(list(cut_words), list(postags))

In [161]:
list(cut_words)

['综合',
 '6',
 '项',
 '指标',
 '，',
 '最后',
 '评',
 '出',
 '工行',
 '综合',
 '服务',
 '最',
 '好',
 '，',
 '北京',
 '银行',
 '最',
 '差',
 '。',
 '北京',
 '银行',
 '之所以',
 '得分',
 '最低',
 '，',
 '主因',
 '是',
 '还',
 '处于',
 '起步',
 '阶段',
 '，',
 '地址',
 '三',
 '街',
 '玉苍路',
 '路口',
 '红绿灯',
 '这',
 '一带',
 '如果',
 '有',
 '看见',
 '的',
 '朋友',
 '请',
 '联系',
 '我',
 '，',
 '手机',
 '号码',
 '是',
 '18757051248',
 '。',
 '感谢',
 '【',
 '他',
 '透露',
 '才',
 '1',
 '年',
 '该',
 '作品',
 '已',
 '涨',
 '至',
 '600万',
 '元',
 '（',
 '约',
 '120万',
 '元',
 '人民币',
 '）',
 '。',
 '另外',
 '还',
 '收藏',
 '草',
 '间',
 '弥生',
 '、',
 '村上隆',
 '、',
 '答',
 '：',
 '对于',
 '目前',
 '的',
 'DotA',
 '来说',
 '，',
 '这',
 '一',
 '点',
 '非常',
 '困难',
 '，',
 '因为',
 '它',
 '缺乏',
 '游戏',
 '周边',
 '的',
 '服务',
 '，',
 '包括',
 'moma',
 '一直',
 '是',
 '科技',
 '地产',
 '的',
 '标杆',
 '，',
 '在',
 '北京',
 '也',
 '具有',
 '一定',
 '的',
 '代表性',
 '，',
 '昨天',
 '，',
 '国内',
 '最',
 '大',
 '发卡',
 '机构',
 '之一',
 '的',
 '建行',
 '正式',
 '颁布',
 '了',
 '其',
 '信用卡',
 '业务',
 '新',
 '规',
 '：',
 '在',
 '年费',
 '政策',
 '上',
 '，',
 '从',
 '本月',
 '起',
 '，

In [168]:
len(cut_words)

215

In [162]:
list(postags)

['v',
 'm',
 'q',
 'n',
 'wp',
 'nd',
 'v',
 'v',
 'j',
 'v',
 'v',
 'd',
 'a',
 'wp',
 'ns',
 'n',
 'd',
 'a',
 'wp',
 'ns',
 'n',
 'c',
 'v',
 'a',
 'wp',
 'n',
 'v',
 'd',
 'v',
 'v',
 'n',
 'wp',
 'n',
 'm',
 'n',
 'ns',
 'n',
 'n',
 'r',
 'n',
 'c',
 'v',
 'v',
 'u',
 'n',
 'v',
 'v',
 'r',
 'wp',
 'n',
 'n',
 'v',
 'm',
 'wp',
 'v',
 'wp',
 'r',
 'v',
 'd',
 'm',
 'q',
 'r',
 'n',
 'd',
 'v',
 'v',
 'm',
 'q',
 'wp',
 'd',
 'm',
 'q',
 'n',
 'wp',
 'wp',
 'r',
 'd',
 'v',
 'n',
 'nd',
 'n',
 'wp',
 'n',
 'wp',
 'v',
 'wp',
 'p',
 'nt',
 'u',
 'ws',
 'u',
 'wp',
 'r',
 'm',
 'q',
 'd',
 'a',
 'wp',
 'c',
 'r',
 'v',
 'n',
 'n',
 'u',
 'v',
 'wp',
 'v',
 'ws',
 'd',
 'v',
 'n',
 'n',
 'u',
 'n',
 'wp',
 'p',
 'ns',
 'd',
 'v',
 'b',
 'u',
 'n',
 'wp',
 'nt',
 'wp',
 'nl',
 'd',
 'a',
 'n',
 'n',
 'r',
 'u',
 'j',
 'a',
 'v',
 'u',
 'r',
 'n',
 'n',
 'a',
 'n',
 'wp',
 'p',
 'n',
 'n',
 'nd',
 'wp',
 'p',
 'nt',
 'nd',
 'wp',
 'c',
 'v',
 'nl',
 'm',
 'n',
 'wp',
 'v',
 'r',
 'wp',


In [166]:
len(postags)

215

In [163]:
list(netags)

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Ni',
 'E-Ni',
 'O',
 'O',
 'O',
 'B-Ni',
 'E-Ni',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'S-Ns',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'S-Ns',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O'

In [167]:
len(netags)

215

### 数据划分

In [151]:
list_BIOES_train[1]

[(84, '实', 'O'),
 (84, '录', 'O'),
 (84, '：', 'O'),
 (84, '丽', 'B_address'),
 (84, '泽', 'I_address'),
 (84, '金', 'I_address'),
 (84, '融', 'I_address'),
 (84, '商', 'I_address'),
 (84, '务', 'I_address'),
 (84, '区', 'E_address'),
 (84, '发', 'O'),
 (84, '展', 'O'),
 (84, '高', 'O'),
 (84, '峰', 'O'),
 (84, '论', 'O'),
 (84, '坛', 'O'),
 (84, '（', 'O'),
 (84, '组', 'O'),
 (84, '图', 'O'),
 (84, '）', 'O'),
 (84, '也', 'O'),
 (84, '不', 'O'),
 (84, '曾', 'O'),
 (84, '看', 'O'),
 (84, '到', 'O'),
 (84, '这', 'O'),
 (84, '两', 'O'),
 (84, '家', 'O'),
 (84, '网', 'O'),
 (84, '站', 'O'),
 (84, '提', 'O'),
 (84, '起', 'O'),
 (84, '过', 'O'),
 (84, '，', 'O'),
 (84, '因', 'O'),
 (84, '此', 'O'),
 (84, '我', 'O'),
 (84, '们', 'O'),
 (84, '可', 'O'),
 (84, '以', 'O'),
 (84, '断', 'O'),
 (84, '定', 'O'),
 (84, '这', 'O'),
 (84, '个', 'O'),
 (84, '倒', 'O'),
 (84, '计', 'O'),
 (84, '时', 'O'),
 (84, '一', 'O'),
 (84, '定', 'O'),
 (84, '是', 'O'),
 (84, '与', 'O'),
 (84, '《', 'B_game'),
 (84, '星', 'I_game'),
 (84, '际', 'I_game'),
 (84, '争', 

In [27]:
train_data_01, test_data_01 = list_BIOES_train[:int(0.8 * len(list_BIOES_train))], list_BIOES_train[int(0.8 * len(list_BIOES_train)):]

In [147]:
len(test_data_01)

503

In [148]:
len(train_data_01)

2012

### 特征生成

In [165]:
df_train[df_train['ID']=='10']

,ID,data
454,10,网络版《大航海时代online》与2004年登场于pc，资料片“南十字星”更新后，“最关键的...


In [28]:
ltp = Ltp(pos=True, ner=True, parse=True)

# string = df_train[df_train['ID']==idx]

def get_other_features(string):
    "对文本进行第三方工具的分词"
    pos_list = []
    ner_list = []
    
    cut_words = list(ltp.segmentor.segment(string))
    postags = list(ltp.postagger.postag(cut_words))
    netags = list(ltp.ner.recognize(cut_words, postags))
    
    for word, pos, ne in zip(cut_words, postags, netags):
        if len(word) > 1:
            for w in word:
                pos_list.append(pos)
                ner_list.append(ne)
        else:
            pos_list.append(pos)
            ner_list.append(ne)
    
    return pos_list, ner_list

In [176]:
pos_list, ner_list = get_other_features(df_train[df_train['ID']=="12"].data.values[0])

In [30]:
def word2features(sent, i):
    "sent每一行为： (id, word, tag), pos, ne_of_ltp"
    word = sent[i][0][1]
    pos = sent[i][1]
    ne = sent[i][2]
    
    features = {
        'bias': 1.0,
        'word': word,
        'word.isdigit()': word.isdigit(),
        'word.istitle()': word.istitle(),
        'word.lower()': word.lower(),
        'pos': pos,
        'pos[:1]': pos[:1],
        'ne': ne,
        'ne[:1]': ne[:1]
    }
    
    if i > 0:
        word = sent[i-1][0][1]
        pos = sent[i-1][1]
        ne = sent[i-1][2]
        features.update({
            '-1:word': word,
            '-1:word.lower()': word.lower(),
            '-1:word.istitle()': word.istitle(),
            '-1:pos': pos,
            '-1:pos[:1]': pos[:1],
            '-1:ne': ne,
            '-1:ne[:1]': ne[:1]
        })
    else:
        features['BOS'] = True

    if i > 1:
        word = sent[i-2][0][1]
        pos = sent[i-2][1]
        ne = sent[i-2][2]
        features.update({
            '-2:word': word,
            '-2:word.lower()': word.lower(),
            '-2:word.istitle()': word.istitle(),
            '-2:pos': pos,
            '-2:pos[:1]': pos[:1],
            '-2:ne': ne,
            '-2:ne[:1]': ne[:1]
        })
    else:
        features['BOS'] = True
        
    if i > 2:
        word = sent[i-3][0][1]
        pos = sent[i-3][1]
        ne = sent[i-3][2]
        features.update({
            '-3:word': word,
            '-3:word.lower()': word.lower(),
            '-3:word.istitle()': word.istitle(),
            '-3:pos': pos,
            '-3:pos[:1]': pos[:1],
            '-3:ne': ne,
            '-3:ne[:1]': ne[:1]
        })
    else:
        features['BOS'] = True
        
    if i > 3:
        word = sent[i-4][0][1]
        pos = sent[i-4][1]
        ne = sent[i-4][2]
        features.update({
            '-4:word': word,
            '-4:word.lower()': word.lower(),
            '-4:word.istitle()': word.istitle(),
            '-4:pos': pos,
            '-4:pos[:1]': pos[:1],
            '-4:ne': ne,
            '-4:ne[:1]': ne[:1]
        })
    else:
        features['BOS'] = True
    
    if i < len(sent)-1:
        word = sent[i+1][0][1]
        pos = sent[i+1][1]
        ne = sent[i+1][2]
        features.update({
            '+1:word': word,
            '+1:word.lower()': word.lower(),
            '+1:word.istitle()': word.istitle(),
            '+1:pos': pos,
            '+1:pos[:1]': pos[:1],
            '+1:ne': ne,
            '+1:ne[:1]': ne[:1]
        })
    else:
        features['EOS'] = True

    if i < len(sent)-2:
        word = sent[i+2][0][1]
        pos = sent[i+2][1]
        ne = sent[i+2][2]
        features.update({
            '+2:word': word,
            '+2:word.lower()': word.lower(),
            '+2:word.istitle()': word.istitle(),
            '+2:pos': pos,
            '+2:pos[:1]': pos[:1],
            '+2:ne': ne,
            '+2:ne[:1]': ne[:1]
        })
    else:
        features['EOS'] = True
        
    if i < len(sent)-3:
        word = sent[i+3][0][1]
        pos = sent[i+3][1]
        ne = sent[i+3][2]
        features.update({
            '+3:word': word,
            '+3:word.lower()': word.lower(),
            '+3:word.istitle()': word.istitle(),
            '+3:pos': pos,
            '+3:pos[:1]': pos[:1],
            '+3:ne': ne,
            '+3:ne[:1]': ne[:1]
        })
    else:
        features['EOS'] = True
        
    if i < len(sent)-4:
        word = sent[i+4][0][1]
        pos = sent[i+4][1]
        ne = sent[i+4][2]
        features.update({
            '+4:word': word,
            '+4:word.lower()': word.lower(),
            '+4:word.istitle()': word.istitle(),
            '+4:pos': pos,
            '+4:pos[:1]': pos[:1],
            '+4:ne': ne,
            '+4:ne[:1]': ne[:1]
        })
    else:
        features['EOS'] = True
        
    return features

In [31]:
wronglist = []

In [32]:
def sent2features(sent):
    idx = sent[0][0]
    pos_list, ner_list = get_other_features(df_train[df_train['ID']==str(idx)].data.values[0])
    # 检查
    if (len(sent) != len(pos_list) or len(pos_list) != len(ner_list) or len(sent) != len(ner_list)):
        print(len(sent))
        print(len(pos_list))
        print(len(ner_list))
        print(idx)
        wronglist.append(str(idx))
        print("========")
    
    sent = list(zip(sent, pos_list, ner_list))
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for ID, token, label in sent]

def sent2tokens(sent):
    return [token for ID, token, label in sent]

In [33]:
%%time
X_train = [sent2features(s) for s in train_data_01]
y_train = [sent2labels(s) for s in train_data_01]

X_test = [sent2features(s) for s in test_data_01]
y_test = [sent2labels(s) for s in test_data_01]

107
105
105
2306
74
70
70
672
86
85
85
1026
95
94
94
46
95
94
94
655
75
74
74
57
Wall time: 15.1 s


In [34]:
wronglist

['2306', '672', '1026', '46', '655', '57']

In [35]:
%%time
X_train = [sent2features(s) for s in train_data_01 if str(s[0][0]) not in wronglist]
y_train = [sent2labels(s) for s in train_data_01 if str(s[0][0]) not in wronglist]

X_test = [sent2features(s) for s in test_data_01 if str(s[0][0]) not in wronglist]
y_test = [sent2labels(s) for s in test_data_01 if str(s[0][0]) not in wronglist]

Wall time: 14.7 s


#### \u3000和\t导致的长度偏差

In [210]:
df_train[df_train['ID']==str(2306)].data.values[0]

'作曲家とくP[阿部尚徳]\u3000基本上会用日语更新微博\u3000GARNiDELiA[w/zMARiA]，VocaloidP[tokuP]，Headphone-Tokyo，日本音乐制作人、GARNiDELiA组合成员Toku-p'

In [211]:
df_train[df_train['ID']==str(1026)].data.values[0]

'2005年超级女声。北京军区空军政治部文工团。北京鸟人文化传播公司，工作联系杨顺利：18901110907，朱妍\t歌手，05超级女声全国10强，首张专辑《你是我爱的风景线》'

In [212]:
df_train[df_train['ID']==str(46)].data.values[0]

'☆LM.C是由主唱maya與吉他手Aiji組成的日本視覺系雙人組合☆\u3000日本官方網站：http://www.lovely-mocochang.com/index.html，日本人气乐团LM.C'

In [213]:
df_train[df_train['ID']==str(672)].data.values[0]

'歌手\u3000主持人\u3000演員\u3000坣瑜伽創辦人\u3000演出經紀請找Lolla：lolla3112@gmail.com，台湾知名歌手，演员坣娜，发行个人专辑《告别临界》等'

In [214]:
df_train[df_train['ID']==str(655)].data.values[0]

'演员。代表作：《美人心计》《生死大营救》《倾城雪》《铁血男儿》《乱世佳人》工作洽谈邮箱：suqing75@163.com胡先生186-1179-4208，苏青\t演员，《美人心计》中饰演大张嫣'

In [215]:
df_train[df_train['ID']==str(57)].data.values[0]

'广东宋祖德影视文化传播有限公司\u3000董事长＼全国第九届青联委员＼世界杰人会香港总会名誉会长＼广东省私营企业协会副会长＼广州市第十届政协常委＼诗人，新浪名博'

In [201]:
# y_test[0]

### 模型训练

In [232]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=200,
    all_possible_transitions=True
)

crf.fit(X_train, y_train)

Wall time: 12min 22s


CRF(algorithm='lbfgs', all_possible_states=None,
  all_possible_transitions=True, averaging=None, c=None, c1=0.1, c2=0.1,
  calibration_candidates=None, calibration_eta=None,
  calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=200,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

#### labels

In [235]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B_company',
 'E_company',
 'I_company',
 'B_address',
 'I_address',
 'E_address',
 'B_book',
 'I_book',
 'E_book',
 'B_game',
 'I_game',
 'E_game',
 'B_organization',
 'I_organization',
 'E_organization',
 'B_scene',
 'I_scene',
 'E_scene',
 'B_name',
 'E_name',
 'B_position',
 'E_position',
 'B_movie',
 'I_movie',
 'E_movie',
 'I_name',
 'I_position',
 'B_government',
 'E_government',
 'B_email',
 'I_email',
 'E_email',
 'I_government',
 'B_mobile',
 'I_mobile',
 'E_mobile',
 'S_company',
 'B_vx',
 'I_vx',
 'E_vx',
 'B_QQ',
 'I_QQ',
 'E_QQ',
 'S_movie',
 'S_name']

#### f1得分

In [236]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=labels)

d:\programdata\envs\nlplearning\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
d:\programdata\envs\nlplearning\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.6829326453299007

In [237]:
y_pred = crf.predict(X_train)
metrics.flat_f1_score(y_train, y_pred, average='weighted', labels=labels)

0.9909461649324396

In [238]:
# y_train[0]
# y_pred[0]

In [239]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

y_pred = crf.predict(X_test)

print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

d:\programdata\envs\nlplearning\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
d:\programdata\envs\nlplearning\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                precision    recall  f1-score   support

          B_QQ      0.000     0.000     0.000         3
          E_QQ      0.000     0.000     0.000         3
          I_QQ      0.000     0.000     0.000        21
     B_address      0.612     0.571     0.591       406
     E_address      0.528     0.493     0.510       406
     I_address      0.686     0.616     0.649      1239
        B_book      0.603     0.537     0.568       136
        E_book      0.620     0.551     0.584       136
        I_book      0.624     0.541     0.580       678
     B_company      0.699     0.647     0.672       405
     E_company      0.676     0.627     0.650       405
     I_company      0.709     0.654     0.681      1404
     S_company      1.000     0.667     0.800         9
       B_email      0.828     0.815     0.822        65
       E_email      0.887     0.833     0.859        66
       I_email      0.903     0.933     0.918      1152
        B_game      0.835     0.734     0.781  

## lgb 模型

### feature生成

In [240]:
def get_data_label(BIOES_data):
    ID = []
    data = []
    label = []
    for s in tqdm(BIOES_data):
        if str(s[0][0]) in wronglist:
            continue
        ID += [s[0][0]] * len(s)
        data += sent2features(s)
        label += sent2labels(s)
    df_data_label = pd.DataFrame(data)
    df_data_label['ID'] = ID
    df_data_label['label'] = label
    return df_data_label

In [241]:
df_train_label_01 = get_data_label(train_data_01)
df_test_label_01 = get_data_label(test_data_01)

100%|█████████████████████████████████████| 503/503 [00:02<00:00, 195.30it/s]


In [242]:
df_train_label_01

,bias,word,word.isdigit(),word.istitle(),word.isupper(),word.lower(),pos,pos[:1],ne,ne[:1],...,-2:word.lower(),-2:word.istitle(),-2:word.isupper(),-2:pos,-2:pos[:1],-2:ne,-2:ne[:1],EOS,ID,label
0,1.0,综,False,False,False,综,v,v,O,O,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2345,O
1,1.0,合,False,False,False,合,v,v,O,O,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2345,O
2,1.0,6,True,False,False,6,m,m,O,O,...,综,False,False,v,v,O,O,NaN,2345,O
3,1.0,项,False,False,False,项,q,q,O,O,...,合,False,False,v,v,O,O,NaN,2345,O
4,1.0,指,False,False,False,指,n,n,O,O,...,6,False,False,m,m,O,O,NaN,2345,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262832,1.0,公,False,False,False,公,n,n,E-Ni,E,...,影,False,False,b,b,I-Ni,I,NaN,758,I_company
262833,1.0,司,False,False,False,司,n,n,E-Ni,E,...,视,False,False,b,b,I-Ni,I,NaN,758,E_company
262834,1.0,董,False,False,False,董,n,n,O,O,...,公,False,False,n,n,E-Ni,E,NaN,758,B_position
262835,1.0,事,False,False,False,事,n,n,O,O,...,司,False,False,n,n,E-Ni,E,True,758,I_position


### 处理数据

In [254]:
features_columns = [c for c in df_train_label_01.columns if c not in ['ID','label']]
label_columns = 'label'

In [255]:
# df_train_label_01_temp = df_train_label_01.head(10000)
# df_train_label_01

In [256]:
# 转为数值特征
for col in tqdm(features_columns + [label_columns]):
    set_col = set(df_train_label_01[col])
    dict_map = dict(zip(set_col, range(len(set_col))))
    df_train_label_01[col] = df_train_label_01[col].map(dict_map)

100%|████████████████████████████████████████| 45/45 [00:04<00:00, 10.79it/s]


In [257]:
# 标签类别
dict(zip(set(df_test_label_01['label']),range(len(set(df_test_label_01['label'])))))

{'B_government': 0,
 'E_QQ': 1,
 'B_scene': 2,
 'E_movie': 3,
 'B_mobile': 4,
 'I_mobile': 5,
 'B_position': 6,
 'I_QQ': 7,
 'E_position': 8,
 'E_government': 9,
 'E_mobile': 10,
 'B_book': 11,
 'S_address': 12,
 'E_game': 13,
 'E_vx': 14,
 'I_movie': 15,
 'B_email': 16,
 'B_vx': 17,
 'I_vx': 18,
 'E_organization': 19,
 'B_name': 20,
 'B_address': 21,
 'E_address': 22,
 'B_game': 23,
 'B_company': 24,
 'I_game': 25,
 'O': 26,
 'I_organization': 27,
 'I_position': 28,
 'S_name': 29,
 'I_government': 30,
 'S_company': 31,
 'I_email': 32,
 'E_email': 33,
 'B_movie': 34,
 'I_address': 35,
 'B_QQ': 36,
 'I_company': 37,
 'I_name': 38,
 'I_scene': 39,
 'B_organization': 40,
 'E_company': 41,
 'I_book': 42,
 'E_scene': 43,
 'E_book': 44,
 'E_name': 45}

In [258]:
df_train_label_01.head()

,bias,word,word.isdigit(),word.istitle(),word.isupper(),word.lower(),pos,pos[:1],ne,ne[:1],...,-2:word.lower(),-2:word.istitle(),-2:word.isupper(),-2:pos,-2:pos[:1],-2:ne,-2:ne[:1],EOS,ID,label
0,0,2560,0,0,0,2539,21,2,9,3,...,0,0,0,0,0,0,0,0,2345,26
1,0,994,0,0,0,985,21,2,9,3,...,0,0,0,0,0,0,0,0,2345,26
2,0,1463,1,0,0,1450,20,0,9,3,...,2526,1,1,22,15,10,4,0,2345,26
3,0,1013,0,0,0,1004,23,7,9,3,...,980,1,1,22,15,10,4,0,2345,26
4,0,3649,0,0,0,3622,19,17,9,3,...,1444,1,1,21,14,10,4,0,2345,26


### 训练模型

In [259]:
from sklearn.model_selection import train_test_split
import lightgbm as lgb

In [260]:
X, y = df_train_label_01[features_columns].values, df_train_label_01['label'].values

In [263]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [264]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 9,
    'learning_rate': 0.3,
    'feature_fraction_seed': 2,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_data': 20,
    'min_hessian': 1,
    'verbose': -1,
    'silent': 0
    }

lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

gbm = lgb.train(
            params,
            lgb_train,
            num_boost_round=10000,
            valid_sets=lgb_eval,
            early_stopping_rounds=20
            )

d:\programdata\envs\nlplearning\lib\site-packages\lightgbm\basic.py:842: UserWarning: silent keyword has been found in `params` and will be ignored.
Please use silent argument of the Dataset constructor to pass this parameter.
  .format(key))


[1]	valid_0's auc: 0.822376
Training until validation scores don't improve for 20 rounds
[2]	valid_0's auc: 0.65617
[3]	valid_0's auc: 0.660165
[4]	valid_0's auc: 0.66504
[5]	valid_0's auc: 0.671839
[6]	valid_0's auc: 0.670474
[7]	valid_0's auc: 0.666769
[8]	valid_0's auc: 0.671061
[9]	valid_0's auc: 0.665647
[10]	valid_0's auc: 0.680099
[11]	valid_0's auc: 0.82902
[12]	valid_0's auc: 0.774469
[13]	valid_0's auc: 0.824265
[14]	valid_0's auc: 0.814663
[15]	valid_0's auc: 0.809568
[16]	valid_0's auc: 0.816434
[17]	valid_0's auc: 0.821303
[18]	valid_0's auc: 0.82265
[19]	valid_0's auc: 0.829351
[20]	valid_0's auc: 0.835341
[21]	valid_0's auc: 0.830098
[22]	valid_0's auc: 0.841381
[23]	valid_0's auc: 0.840242
[24]	valid_0's auc: 0.844463
[25]	valid_0's auc: 0.848978
[26]	valid_0's auc: 0.851743
[27]	valid_0's auc: 0.85189
[28]	valid_0's auc: 0.852851
[29]	valid_0's auc: 0.853838
[30]	valid_0's auc: 0.868208
[31]	valid_0's auc: 0.870056
[32]	valid_0's auc: 0.874046
[33]	valid_0's auc: 0.875

In [271]:
importance = list(zip(gbm.feature_importance(),features_columns))

In [274]:
sorted(importance, key=lambda x: x[0], reverse=True)

[(118, '+1:word'),
 (88, '+2:word'),
 (85, 'ne[:1]'),
 (85, '+1:word.lower()'),
 (80, 'word'),
 (75, 'word.lower()'),
 (58, '+2:word.lower()'),
 (57, '-2:word'),
 (56, '-1:word'),
 (52, '+2:ne'),
 (48, '+1:pos'),
 (47, '+2:ne[:1]'),
 (46, '+1:ne[:1]'),
 (45, '-2:word.lower()'),
 (44, 'ne'),
 (44, '+2:pos'),
 (44, '-1:pos'),
 (42, 'pos'),
 (40, '+1:ne'),
 (40, '-2:pos'),
 (35, '-1:word.lower()'),
 (34, '-1:pos[:1]'),
 (27, '-2:ne'),
 (26, '-1:ne'),
 (20, '+1:pos[:1]'),
 (20, '+2:pos[:1]'),
 (18, 'pos[:1]'),
 (16, '-2:pos[:1]'),
 (14, '-1:ne[:1]'),
 (7, '-2:ne[:1]'),
 (1, 'word.istitle()'),
 (1, '+1:word.istitle()'),
 (1, '+2:word.istitle()'),
 (1, '-1:word.istitle()'),
 (1, '-2:word.istitle()'),
 (0, 'bias'),
 (0, 'word.isdigit()'),
 (0, 'word.isupper()'),
 (0, 'BOS'),
 (0, '+1:word.isupper()'),
 (0, '+2:word.isupper()'),
 (0, '-1:word.isupper()'),
 (0, '-2:word.isupper()'),
 (0, 'EOS')]

# 其他第三方工具特征处理

In [494]:
!pip install jieba

In [498]:
import jieba

In [499]:
seg_list = jieba.cut("我来到北京清华大学", cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

seg_list = jieba.cut("我来到北京清华大学", cut_all=False)
print("Default Mode: " + "/ ".join(seg_list))  # 精确模式

seg_list = jieba.cut("他来到了网易杭研大厦")  # 默认是精确模式
print(", ".join(seg_list))

seg_list = jieba.cut_for_search("小明硕士毕业于中国科学院计算所，后在日本京都大学深造")  # 搜索引擎模式
print(", ".join(seg_list))

Full Mode: 我/ 来到/ 北京/ 清华/ 清华大学/ 华大/ 大学
Default Mode: 我/ 来到/ 北京/ 清华大学
他, 来到, 了, 网易, 杭研, 大厦
小明, 硕士, 毕业, 于, 中国, 科学, 学院, 科学院, 中国科学院, 计算, 计算所, ，, 后, 在, 日本, 京都, 大学, 日本京都大学, 深造


In [500]:
seg_list = jieba.cut(df_train[df_train['ID']=='820'].data.values[0], cut_all=True)
print("Full Mode: " + "/ ".join(seg_list))  # 全模式

Full Mode: 我们/ 今天/ 研讨/ 的/ 问题/ 是/ 西部/ 豪宅/ ，/ 我们/ 谈到/ 西部/ 豪宅/ ，/ 必定/ 针对/ 的/ 就是/ 西部/ 客/ 群/ 。/ 无论/ 在/ 西单/ 置业/ 的/ 人/ 也好/ ，/ 甚至/ 还/ 可以/ 和/ 他/ 赌博/ 小/ 玩/ 一把/ 从/ 他/ 身上/ 赢取/ 金钱/ 。/ 可以/ 肯定/ 的/ 是/ ，/ 在/ 《/ 怒潮/ 》/ 现阶段/ 阶段/ 是/ 对/ 紫砂/ 紫砂壶/ 砂壶/ 收藏/ 与/ 投资/ 的/ 一个/ 好时机/ 时机/ ，/ 在/ 艺术/ 艺术品/ 市场/ 场上/ ，/ 何/ 道/ 洪/ 等/ 紫砂/ 工艺/ 大师/ 的/ 精品/ 备受/ 关注/ ，/ Infi/ 打算/ 顺道/ 将/ 此处/ 的/ 红龙/ 怪/ 练/ 掉/ ，/ 但/ 只/ 凭/ 大法/ 大法师/ 法师/ +/ 水/ 元素/ 两个/ 对/ 空/ 虽说/ 也/ 能/ 杀掉/ 红龙/ ，/ 学生/ elaine/ 说/ ，/ 25/ 日/ 晚/ ，/ 他们/ 一家/ 三口/ 在/ 文汇/ 路/ 的/ 教室/ 室里/ 等到/ 第二/ 第二天/ 二天/ 凌晨/ 2/ 点/ 才/ 离开/ 。/ 其间/ ，/ 但是/ ，/ 日本/ 防卫/ 省/ 要/ 引进/ f/ -/ 35/ 还/ 须/ 获得/ 日本/ 新政/ 新政府/ 政府/ 的/ 同意/ 。/ 由于/ f/ -/ 35/ 尚/ 处于/ 开发/ 开发阶段/ 阶段/ ，/ D/ 也/ 沉寂/ 了/ 大半/ 大半年/ 半年/ 年后/ 终于/ 耐不住/ 不住/ 寂寞/ ，/ 错/ ，/ 是/ 对/ 职业/ Dota/ 的/ 向往/ ，/ 他/ 回来/ 了/ ！/ 以下/ 是/ 他/ 的/ 专访/ ：/ 一/ 水/ 横亘/ 环境/ 中/ 的/ 项目/ 是非/ 非常/ 难得/ 的/ ，/ 更何况/ 何况/ 她/ 是/ 在/ 西山/ 山环/ 环抱/ 之中/ ，/ 更/ 凝聚/ 了/ 深厚/ 的/ 历史/ 人文/ 积淀/ 。


In [502]:
seg_list = jieba.cut(df_train[df_train['ID']=='820'].data.values[0])
print("Full Mode: " + " ".join(seg_list))  # 精确模式

Full Mode: 我们 今天 研讨 的 问题 是 西部 豪宅 ， 我们 谈到 西部 豪宅 ， 必定 针对 的 就是 西部 客群 。 无论 在 西单 置业 的 人 也好 ， 甚至 还 可以 和 他 赌博 小玩 一把 从 他 身上 赢取 金钱 。 可以 肯定 的 是 ， 在 《 怒潮 》 现阶段 是 对 紫砂壶 收藏 与 投资 的 一个 好时机 ， 在 艺术品 市场 上 ， 何道 洪等 紫砂 工艺 大师 的 精品 备受 关注 ， Infi 打算 顺道 将 此处 的 红龙 怪练 掉 ， 但 只 凭 大法师 + 水 元素 两个 对空 虽说 也 能 杀掉 红龙 ， 学生 elaine 说 ， 25 日晚 ， 他们 一家 三口 在 文汇 路 的 教室 里 等到 第二天 凌晨 2 点才 离开 。 其间 ， 但是 ， 日本 防卫 省要 引进 f - 35 还须 获得 日本 新政府 的 同意 。 由于 f - 35 尚 处于 开发阶段 ， D 也 沉寂 了 大半年 后 终于 耐不住 寂寞 ， 错 ， 是 对 职业 Dota 的 向往 ， 他 回来 了 ！ 以下 是 他 的 专访 ： 一水 横亘 环境 中 的 项目 是 非常 难得 的 ， 更何况 她 是 在 西山 环抱 之中 ， 更 凝聚 了 深厚 的 历史 人文 积淀 。


In [513]:
df_train[df_train['ID']=='820'].data.values[0]

'我们今天研讨的问题是西部豪宅，我们谈到西部豪宅，必定针对的就是西部客群。无论在西单置业的人也好，甚至还可以和他赌博小玩一把从他身上赢取金钱。可以肯定的是，在《怒潮》现阶段是对紫砂壶收藏与投资的一个好时机，在艺术品市场上，何道洪等紫砂工艺大师的精品备受关注，Infi打算顺道将此处的红龙怪练掉，但只凭大法师+水元素两个对空虽说也能杀掉红龙，学生elaine说，25日晚，他们一家三口在文汇路的教室里等到第二天凌晨2点才离开。其间，但是，日本防卫省要引进f-35还须获得日本新政府的同意。由于f-35尚处于开发阶段，D也沉寂了大半年后终于耐不住寂寞，错，是对职业Dota的向往，他回来了！以下是他的专访：一水横亘环境中的项目是非常难得的，更何况她是在西山环抱之中，更凝聚了深厚的历史人文积淀。'

In [503]:
import jieba.posseg as pseg
words = pseg.cut("我爱北京天安门")  #jieba默认模式
jieba.enable_paddle()  #启动paddle模式。 0.40版之后开始支持，早期版本不支持
words = pseg.cut("我爱北京天安门", use_paddle=True)  #paddle模式
for word, flag in words:
    print('%s %s' % (word, flag))

Paddle enabled successfully......


我 r
爱 v
北京 LOC
天安门 LOC


In [504]:
words = pseg.cut(df_train[df_train['ID']=='820'].data.values[0]) #jieba默认模式
jieba.enable_paddle() #启动paddle模式。 0.40版之后开始支持，早期版本不支持
words = pseg.cut(df_train[df_train['ID']=='820'].data.values[0],use_paddle=True) #paddle模式

for word, flag in words:
    print('%s %s' % (word, flag))

Paddle enabled successfully......


我们 r
今天 TIME
研讨 v
的 u
问题 n
是 v
西部 f
豪宅 n
， v
我们 r
谈到 v
西部 f
豪宅 n
， n
必定 d
针对 v
的 u
就是 v
西部 f
客群 n
。 n
无 c
论 c
在 p
西单置业 ORG
的 u
人 n
也 d
好 d
， v
甚至 d
还 d
可以 v
和 p
他 r
赌博 v
小玩 nr
一把 m
从 p
他 r
身上 s
赢取 v
金钱。 nz
可以 v
肯定 v
的 u
是 v
， v
在 p
《 w
怒潮 nw
》 w
现阶段 t
是 v
对 p
紫砂壶 n
收藏 vn
与 c
投资 vn
的 u
一个 m
好时机 n
， v
在 p
艺术品 n
市场 n
上 f
， v
何道洪 PER
等 u
紫砂 n
工艺 n
大师 n
的 u
精品 n
备受 v
关注 v
，Infi nr
打算 v
顺道 d
将 p
此处 s
的 u
红龙 n
怪 v
练 v
掉 v
， n
但 c
只 d
凭 p
大法师 n
+ w
水 n
元素 n
两个 m
对空 vn
虽说 c
也 d
能 v
杀 v
掉 v
红龙 n
， v
学生 n
elaine nr
说 v
， n
25日晚 TIME
， v
他们 r
一家 m
三口 m
在 p
文汇路 LOC
的 u
教室 n
里 f
等到 v
第二天 TIME
凌晨2点 TIME
才 d
离开 v
。 n
其间 f
， n
但是 c
， v
日本防卫省 LOC
要 v
引进 v
f xc
- w
35 m
还须 d
获得 v
日本 LOC
新 a
政府 n
的 u
同意 vn
。 n
由于 p
f xc
- w
35 m
尚 d
处于 v
开发 vn
阶段 n
，D nr
也 d
沉寂 a
了 u
大 a
半年 TIME
后 f
终于 d
耐 v
不住 v
寂寞 an
， v
错 a
， n
是 v
对 p
职业 n
Dota nr
的 u
向往 vn
， v
他 r
回来 v
了 u
！ n
以下 f
是 v
他 r
的 u
专访 vn
： n
一 m
水 n
横亘 v
环境 n
中 f
的 u
项目 n
是 v
非常 d
难得 a
的 u
， n
更何况 d
她 r
是 v
在 p
西山 LOC
环抱 v
之 u
中， nz
更 d
凝聚 v
了 u
深厚 a
的 u


In [511]:
list_text_tags = []
for word, flag in words:
    for w in list(word):
        list_text_tags += [(w, flag)]

In [234]:
# list_text_tags

In [406]:
df_label_train[df_label_train['ID']==820]

,ID,Category,Pos_b,Pos_e,Privacy
0,820,address,39,40,西单
1,820,game,78,81,《怒潮》
2,820,name,110,112,何道洪
3,820,position,114,119,紫砂工艺大师
4,820,name,128,131,Infi
5,820,address,191,193,文汇路
6,820,name,171,176,elaine
7,820,position,169,170,学生
8,820,government,217,221,日本防卫省
9,820,government,233,237,日本新政府


In [506]:
result = jieba.tokenize(df_train[df_train['ID']=='820'].data.values[0], mode='search')
for tk in result:
    print("word %s\t\t start: %d \t\t end:%d" % (tk[0],tk[1],tk[2]))

word 我们		 start: 0 		 end:2
word 今天		 start: 2 		 end:4
word 研讨		 start: 4 		 end:6
word 的		 start: 6 		 end:7
word 问题		 start: 7 		 end:9
word 是		 start: 9 		 end:10
word 西部		 start: 10 		 end:12
word 豪宅		 start: 12 		 end:14
word ，		 start: 14 		 end:15
word 我们		 start: 15 		 end:17
word 谈到		 start: 17 		 end:19
word 西部		 start: 19 		 end:21
word 豪宅		 start: 21 		 end:23
word ，		 start: 23 		 end:24
word 必定		 start: 24 		 end:26
word 针对		 start: 26 		 end:28
word 的		 start: 28 		 end:29
word 就是		 start: 29 		 end:31
word 西部		 start: 31 		 end:33
word 客群		 start: 33 		 end:35
word 。		 start: 35 		 end:36
word 无论		 start: 36 		 end:38
word 在		 start: 38 		 end:39
word 西单		 start: 39 		 end:41
word 置业		 start: 41 		 end:43
word 的		 start: 43 		 end:44
word 人		 start: 44 		 end:45
word 也好		 start: 45 		 end:47
word ，		 start: 47 		 end:48
word 甚至		 start: 48 		 end:50
word 还		 start: 50 		 end:51
word 可以		 start: 51 		 end:53
word 和		 start: 53 		 end:54
word 他		 start: 54 		 end:55
word